# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-NOTEBOOK_CODE_HERE-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the common aspects between movies directed by Woody Allen or Quentin Tarantino. We want to know the people that worked for both directors with some numerical analyses, what are the differences in terms of budget for their movies, who won more Academy Awards. 

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q2526255`| Director       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`    | Quentin Tarantino      | node |





Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**


The workload should:


1. Identify the BGP for films

2. Identify the BGP for directors

3. Identify the BGP for workers in a films

4. Compare the workers amongst the films directed by the two directors

5. Return some numerical comparison between the two directors (e.g., how many workers in Tarantino's movies who also worked in Allen's films?, what is the film with the highest number of shared actors? Who is the most used actor by both the directors? etc. )

6. Is the maximum budget for a Tarantino's movie higher of the max budget of an Allen's movie?

7. Who has films with more nominations for Academy Awards and who won more Academy Awards (with his films not only personal awards).

    7.1 Find the BGP for Academy Awards 

    7.2 Find the related subproperties

    7.3 Find how they are related to the directors
    
    7.4 Are there alternative queries to get the same result?



In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT DISTINCT *
WHERE { 

wd:Q25089 ?p ?obj .


} 

LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1006'), ('obj', 'http://data.bibliotheken.nl/id/thes/p06917654X')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1015'), ('obj', 'https://livedata.bibsys.no/authority/90085344')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1150'), ('obj', 'http://rvk.uni-regensburg.de/nt/HU+3027')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1741'), ('obj', 'http://data.beeldengeluid.nl/gtaa/76199')]


5

# Task 1

In [7]:
# 1. Identify the BGP for films

In [ ]:
# I look for all human directors 

In [15]:
queryString = """
SELECT DISTINCT ?d ?dname
WHERE { 

?d   wdt:P106 wd:Q2526255;
     wdt:P31  wd:Q5 .

# this returns the labels
    ?d <http://schema.org/name> ?dname .

}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('d', 'http://www.wikidata.org/entity/Q11562694'), ('dname', 'Takayoshi Watanabe')]
[('d', 'http://www.wikidata.org/entity/Q64006834'), ('dname', 'Lois Patiño')]
[('d', 'http://www.wikidata.org/entity/Q8186437'), ('dname', 'Buzz Feitshans IV')]
[('d', 'http://www.wikidata.org/entity/Q8195115'), ('dname', 'Alfonso Arandia')]
[('d', 'http://www.wikidata.org/entity/Q8203028'), ('dname', 'Arantza Ibarra')]
[('d', 'http://www.wikidata.org/entity/Q105319'), ('dname', 'Helmut Herbst')]
[('d', 'http://www.wikidata.org/entity/Q1106742'), ('dname', "Pat O'Connor")]
[('d', 'http://www.wikidata.org/entity/Q1124438'), ('dname', 'Zsolt Kézdi-Kovács')]
[('d', 'http://www.wikidata.org/entity/Q11364257'), ('dname', 'Setsuo Nakayama')]
[('d', 'http://www.wikidata.org/entity/Q11552553'), ('dname', 'Shūichi Okita')]


10

In [12]:
# Now I look for properties for director

In [17]:
queryString = """
SELECT DISTINCT  ?p ?pname
WHERE { 

?d   wdt:P106 wd:Q2526255;
     wdt:P31  wd:Q5 ;
     ?p ?o .

# this returns the labels
    ?d <http://schema.org/name> ?dname .
    ?p <http://schema.org/name> ?pname .

}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6891'), ('pname', 'National Film Board of Canada director identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P5985'), ('pname', 'Literaturaren Zubitegia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pname', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('pname', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('pname', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('pname', 'manner of death')]


10

In [18]:
# Now I want to look for Woody Allen  (wd:Q25089 --> Woody Allen)

In [46]:
queryString = """
SELECT DISTINCT  ?p ?pname
WHERE { 

wd:Q25089 ?p ?o .

# this returns the labels
    ?p <http://schema.org/name> ?pname .

#FILTER REGEX(?pname, "work")
}

ORDER BY ASC (?pname)
LIMIT 20

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6150'), ('pname', 'Academy Awards Database nominee ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5121'), ('pname', 'All About Jazz musician ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2019'), ('pname', 'AllMovie person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1728'), ('pname', 'AllMusic artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7214'), ('pname', 'Allcinema person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('pname', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3782'), ('pname', 'Artnet artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3280'), ('pname', 'BAnQ author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5491'), ('pname', "BD Gest' author ID")]
[('p', 'http://www.wikidata.org/prop/direct/P6656'), ('pname', 'BHCL ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P9037'), ('pname', 'BHCL UUID')]
[('p', 'http://www.wikidata.org

20

In [43]:
# GET: 'occupation' --> wdt:P106
# GET: 'notable work' --> wdt:P800

In [ ]:
# Use 'notable work' to find Woody's works

In [45]:
queryString = """
SELECT DISTINCT  ?w ?wname
WHERE { 


 wd:Q25089 wdt:P800 ?w .        # ?w --> works
 
 
# this returns the labels
    ?w <http://schema.org/name> ?wname .
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('w', 'http://www.wikidata.org/entity/Q206124'), ('wname', 'Midnight in Paris')]
[('w', 'http://www.wikidata.org/entity/Q971865'), ('wname', "What's Up, Tiger Lily?")]
[('w', 'http://www.wikidata.org/entity/Q1004531'), ('wname', 'Bullets Over Broadway')]
[('w', 'http://www.wikidata.org/entity/Q1354109'), ('wname', 'Mighty Aphrodite')]
[('w', 'http://www.wikidata.org/entity/Q729026'), ('wname', 'Broadway Danny Rose')]
[('w', 'http://www.wikidata.org/entity/Q733677'), ('wname', 'Match Point')]
[('w', 'http://www.wikidata.org/entity/Q740143'), ('wname', 'Husbands and Wives')]
[('w', 'http://www.wikidata.org/entity/Q806092'), ('wname', 'Bananas')]
[('w', 'http://www.wikidata.org/entity/Q1321004'), ('wname', 'You Will Meet a Tall Dark Stranger')]
[('w', 'http://www.wikidata.org/entity/Q14511869'), ('wname', 'Magic in the Moonlight')]


10

In [24]:
# Look for Allen's works type

In [47]:
queryString = """
SELECT DISTINCT  ?o ?oname
WHERE { 


 wd:Q25089 wdt:P800 ?w .        # ?w --> works
 
 ?w wdt:P31 ?o .
 
# this returns the labels
    ?w <http://schema.org/name> ?wname .
    ?o <http://schema.org/name> ?oname .
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11424'), ('oname', 'film')]
[('o', 'http://www.wikidata.org/entity/Q5398426'), ('oname', 'television series')]


2

In [48]:
# GET: 'film' --> wd:Q11424

In [49]:
# Now I can find BGP for films

In [51]:
queryString = """
SELECT DISTINCT ?film ?filmname
WHERE { 


 ?film wdt:P31 wd:Q11424 .        
 
 
# this returns the labels
    ?film <http://schema.org/name> ?filmname .
    
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q8178307'), ('filmname', '15 Park Avenue')]
[('film', 'http://www.wikidata.org/entity/Q8197668'), ('filmname', 'Idiot Love')]
[('film', 'http://www.wikidata.org/entity/Q319263'), ('filmname', 'Doctor in the House')]
[('film', 'http://www.wikidata.org/entity/Q8452990'), ('filmname', "Sunday League - Pepik Hnatek's Final Match")]
[('film', 'http://www.wikidata.org/entity/Q284184'), ('filmname', 'Ra.One')]
[('film', 'http://www.wikidata.org/entity/Q13637926'), ('filmname', 'Indra')]
[('film', 'http://www.wikidata.org/entity/Q13638153'), ('filmname', 'Inner Senses')]
[('film', 'http://www.wikidata.org/entity/Q13645807'), ('filmname', 'Madhumasam')]
[('film', 'http://www.wikidata.org/entity/Q13647258'), ('filmname', 'Marupakkam')]
[('film', 'http://www.wikidata.org/entity/Q13677566'), ('filmname', 'Timberjack')]


10

In [52]:
# Per curiosità conto i tutti i film presenti sul database

In [53]:
queryString = """
SELECT DISTINCT count(?film) as ?num_film
WHERE { 


 ?film wdt:P31 wd:Q11424 .        
 
 
# this returns the labels
    ?film <http://schema.org/name> ?filmname .
    
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('num_film', '184122')]


1

In [54]:
#
#

# Task 2

In [ ]:
# 2. Identify the BGP for directors

In [ ]:
# Look for profession of Woody Allen

In [55]:
queryString = """
SELECT DISTINCT  ?o ?oname
WHERE { 


 wd:Q25089 wdt:P106 ?o .        # ?o --> occupations
 

 
# this returns the labels
   
    ?o <http://schema.org/name> ?oname .
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q10800557'), ('oname', 'film actor')]
[('o', 'http://www.wikidata.org/entity/Q15981151'), ('oname', 'jazz musician')]
[('o', 'http://www.wikidata.org/entity/Q18844224'), ('oname', 'science fiction writer')]
[('o', 'http://www.wikidata.org/entity/Q1930187'), ('oname', 'journalist')]
[('o', 'http://www.wikidata.org/entity/Q214917'), ('oname', 'playwright')]
[('o', 'http://www.wikidata.org/entity/Q245068'), ('oname', 'comedian')]
[('o', 'http://www.wikidata.org/entity/Q2526255'), ('oname', 'film director')]
[('o', 'http://www.wikidata.org/entity/Q28389'), ('oname', 'screenwriter')]
[('o', 'http://www.wikidata.org/entity/Q3282637'), ('oname', 'film producer')]
[('o', 'http://www.wikidata.org/entity/Q36180'), ('oname', 'writer')]


10

In [57]:
# GET: 'film director' --> Q2526255

In [ ]:
# BGP to find all film directors 

In [58]:
queryString = """
SELECT DISTINCT  ?fd ?fdname
WHERE { 


 ?fd wdt:P106 wd:Q2526255 ;       # ?fd --> film directors
     wdt:P31  wd:Q5 .
 

 
# this returns the labels
   
    ?fd <http://schema.org/name> ?fdname .
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('fd', 'http://www.wikidata.org/entity/Q11562694'), ('fdname', 'Takayoshi Watanabe')]
[('fd', 'http://www.wikidata.org/entity/Q64006834'), ('fdname', 'Lois Patiño')]
[('fd', 'http://www.wikidata.org/entity/Q8186437'), ('fdname', 'Buzz Feitshans IV')]
[('fd', 'http://www.wikidata.org/entity/Q8195115'), ('fdname', 'Alfonso Arandia')]
[('fd', 'http://www.wikidata.org/entity/Q8203028'), ('fdname', 'Arantza Ibarra')]
[('fd', 'http://www.wikidata.org/entity/Q105319'), ('fdname', 'Helmut Herbst')]
[('fd', 'http://www.wikidata.org/entity/Q1106742'), ('fdname', "Pat O'Connor")]
[('fd', 'http://www.wikidata.org/entity/Q1124438'), ('fdname', 'Zsolt Kézdi-Kovács')]
[('fd', 'http://www.wikidata.org/entity/Q11364257'), ('fdname', 'Setsuo Nakayama')]
[('fd', 'http://www.wikidata.org/entity/Q11552553'), ('fdname', 'Shūichi Okita')]


10

In [ ]:
#
#

# Task 3

In [59]:
# 3. Identify the BGP for workers in a films

In [64]:
queryString = """
SELECT DISTINCT  ?pro ?proname
WHERE { 


 ?w wdt:P106 ?pro ;       # ?fd --> film directors
    wdt:P31  wd:Q5 ;
    ?p ?film . 
 ?film wdt:P31 wd:Q11424 . 

 
# this returns the labels
  #  ?p <http://schema.org/name> ?pname .
    ?pro <http://schema.org/name> ?proname .
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('pro', 'http://www.wikidata.org/entity/Q12809351'), ('proname', 'The Bread Seller Woman')]
[('pro', 'http://www.wikidata.org/entity/Q497240'), ('proname', 'master of ceremonies')]
[('pro', 'http://www.wikidata.org/entity/Q191633'), ('proname', 'mangaka')]
[('pro', 'http://www.wikidata.org/entity/Q17456089'), ('proname', 'pornographic film director')]
[('pro', 'http://www.wikidata.org/entity/Q10798782'), ('proname', 'television actor')]
[('pro', 'http://www.wikidata.org/entity/Q10800557'), ('proname', 'film actor')]
[('pro', 'http://www.wikidata.org/entity/Q121998'), ('proname', 'ambassador')]
[('pro', 'http://www.wikidata.org/entity/Q1414443'), ('proname', 'filmmaker')]
[('pro', 'http://www.wikidata.org/entity/Q177220'), ('proname', 'singer')]
[('pro', 'http://www.wikidata.org/entity/Q18844224'), ('proname', 'science fiction writer')]


10

In [ ]:
# Per curiosità conto i tutti i tipi di lavoratori connessi ai film presenti sul database

In [4]:
queryString = """
SELECT DISTINCT  count( distinct ?pro) as ?num_professions
WHERE { 


 ?w wdt:P106 ?pro ;       # ?fd --> film directors
    wdt:P31  wd:Q5 ;
    ?p ?film . 
 ?film wdt:P31 wd:Q11424 . 

 
# this returns the labels
  #  ?p <http://schema.org/name> ?pname .
    ?pro <http://schema.org/name> ?proname .
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('num_professions', '752')]


1

In [ ]:
#
#

# Task 4

In [ ]:
# 4. Compare the workers amongst the films directed by the two directors

In [6]:
# GET:
# wd:Q25089	Woody Allen
# wd:Q3772	Quentin Tarantino
# wd:Q2526255   film director

In [7]:
# I want to find the property that connect a film to a director

In [4]:
queryString = """
SELECT DISTINCT  ?p ?pname
WHERE { 


 ?filmwa wdt:P31 wd:Q11424 .    # ?filmwa --> film Woody Allen
 ?filmwa ?p  wd:Q25089.        


# this returns the labels
    ?p <http://schema.org/name> ?pname .
    
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('pname', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pname', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('pname', 'screenwriter')]
[('p', 'http://www.wikidata.org/prop/direct/P86'), ('pname', 'composer')]


5

In [7]:
# GET: 'director' --> wdt:P57

In [ ]:
# Look for Woody Allen type of workers

In [12]:
queryString = """
SELECT DISTINCT  ?proWA ?pronameWA
WHERE { 


 ?w wdt:P106 ?proWA ;       # ?fd --> film directors
    wdt:P31  wd:Q5 .
    ?filmwa ?p ?w . 
{ 
SELECT DISTINCT  ?filmwa
WHERE { 

 ?filmwa wdt:P31 wd:Q11424 .    # ?filmwa --> film Woody Allen
 ?filmwa wdt:P57 wd:Q25089 .        
}
}  
# this returns the labels
  #  ?p <http://schema.org/name> ?pname .
    ?proWA <http://schema.org/name> ?pronameWA .
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('proWA', 'http://www.wikidata.org/entity/Q11075015'), ('pronameWA', 'war criminal')]
[('proWA', 'http://www.wikidata.org/entity/Q144662'), ('pronameWA', 'sports agent')]
[('proWA', 'http://www.wikidata.org/entity/Q18581305'), ('pronameWA', 'beauty pageant contestant')]
[('proWA', 'http://www.wikidata.org/entity/Q2707485'), ('pronameWA', 'scenographer')]
[('proWA', 'http://www.wikidata.org/entity/Q2962070'), ('pronameWA', 'production designer')]
[('proWA', 'http://www.wikidata.org/entity/Q713200'), ('pronameWA', 'performing artist')]
[('proWA', 'http://www.wikidata.org/entity/Q1062952'), ('pronameWA', 'character designer')]
[('proWA', 'http://www.wikidata.org/entity/Q1823479'), ('pronameWA', 'lighting designer')]
[('proWA', 'http://www.wikidata.org/entity/Q1776724'), ('pronameWA', 'theatre manager')]
[('proWA', 'http://www.wikidata.org/entity/Q911554'), ('pronameWA', 'business magnate')]
[('proWA', 'http://www.wikidata.org/entity/Q19509201'), ('pronameWA', 'LGBTQIA+ rights act

214

In [ ]:
# Look for Quentin Tarantino type of workers

In [11]:
queryString = """
SELECT DISTINCT  ?proQT ?pronameQT
WHERE { 


 ?w wdt:P106 ?proQT ;       # ? --> film directors
    wdt:P31  wd:Q5 .
    ?filmqt ?p ?w. 
{ 
SELECT DISTINCT  ?filmqt
WHERE { 

 ?filmqt wdt:P31 wd:Q11424 .    # ?filmqt --> film Quentin Tarantino
 ?filmqt wdt:P57 wd:Q3772.        
}
}  
# this returns the labels
  #  ?p <http://schema.org/name> ?pname .
    ?proQT <http://schema.org/name> ?pronameQT .
}

LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('proQT', 'http://www.wikidata.org/entity/Q18581305'), ('pronameQT', 'beauty pageant contestant')]
[('proQT', 'http://www.wikidata.org/entity/Q378622'), ('pronameQT', 'racing driver')]
[('proQT', 'http://www.wikidata.org/entity/Q622807'), ('pronameQT', 'seiyū')]
[('proQT', 'http://www.wikidata.org/entity/Q1643514'), ('pronameQT', 'music arranger')]
[('proQT', 'http://www.wikidata.org/entity/Q154549'), ('pronameQT', 'carpenter')]
[('proQT', 'http://www.wikidata.org/entity/Q10264925'), ('pronameQT', 'demagogue')]
[('proQT', 'http://www.wikidata.org/entity/Q19509201'), ('pronameQT', 'LGBTQIA+ rights activist')]
[('proQT', 'http://www.wikidata.org/entity/Q107711'), ('pronameQT', 'firefighter')]
[('proQT', 'http://www.wikidata.org/entity/Q13561328'), ('pronameQT', 'surfer')]
[('proQT', 'http://www.wikidata.org/entity/Q3922505'), ('pronameQT', 'music producer')]
[('proQT', 'http://www.wikidata.org/entity/Q1028181'), ('pronameQT', 'painter')]
[('proQT', 'http://www.wikidata.org/entit

118

In [ ]:
# For Woody Allen there are 214 different type of workers
# For Quentin Tarantino there are 118 different type of workers

# Task 5

In [76]:
# 5. Return some numerical comparison between the two directors (e.g., how many workers in Tarantino's movies 
#    who also worked in Allen's films?, what is the film with the highest number of shared actors? Who is the most
#    used actor by both the directors? etc. )

In [11]:
# how many workers in Tarantino's movies who also worked in Allen's films?

In [14]:
# Now I'm intersted to have the name of the workers that work for both the directors

In [13]:
queryString = """
SELECT DISTINCT  ?ww ?wwname
WHERE {

{
SELECT DISTINCT  ?ww ?wwname
WHERE { 
 ?ww wdt:P106 ?proWA ;       # ?fd --> film directors
     wdt:P31  wd:Q5 .
   ?filmwa ?p  ?ww.  
{ 
SELECT DISTINCT  ?filmwa
WHERE { 

 ?filmwa wdt:P31 wd:Q11424 .    # ?filmwa --> film Woody Allen
 ?filmwa wdt:P57 wd:Q25089 .        
}
}  
# this returns the labels
    ?ww <http://schema.org/name> ?wwname .
   # ?proWA <http://schema.org/name> ?pronameWA .
}
}

{
SELECT DISTINCT  ?wq ?wqname
WHERE { 


 ?wq wdt:P106 ?proQT ;       # ? --> film directors
     wdt:P31  wd:Q5 .
   ?filmqt ?p  ?wq. 
{ 
SELECT DISTINCT  ?filmqt
WHERE { 

 ?filmqt wdt:P31 wd:Q11424 .    # ?filmqt --> film Quentin Tarantino
 ?filmqt wdt:P57 wd:Q3772.        
}
}  
# this returns the labels
    ?wq <http://schema.org/name> ?wqname .
   # ?proQT <http://schema.org/name> ?pronameQT .
}
}

FILTER(?wq = ?ww)
}


"""

print("Results")
run_query(queryString)

Results
[('ww', 'http://www.wikidata.org/entity/Q185724'), ('wwname', 'Mike Myers')]
[('ww', 'http://www.wikidata.org/entity/Q104061'), ('wwname', 'Steve Buscemi')]
[('ww', 'http://www.wikidata.org/entity/Q231096'), ('wwname', 'Léa Seydoux')]
[('ww', 'http://www.wikidata.org/entity/Q109232'), ('wwname', 'Rosanna Arquette')]
[('ww', 'http://www.wikidata.org/entity/Q207596'), ('wwname', 'Daryl Hannah')]
[('ww', 'http://www.wikidata.org/entity/Q352'), ('wwname', 'Adolf Hitler')]
[('ww', 'http://www.wikidata.org/entity/Q26806'), ('wwname', 'Danny DeVito')]
[('ww', 'http://www.wikidata.org/entity/Q36949'), ('wwname', 'Robert De Niro')]
[('ww', 'http://www.wikidata.org/entity/Q125017'), ('wwname', 'Uma Thurman')]
[('ww', 'http://www.wikidata.org/entity/Q38111'), ('wwname', 'Leonardo DiCaprio')]
[('ww', 'http://www.wikidata.org/entity/Q41749'), ('wwname', 'Joseph Goebbels')]
[('ww', 'http://www.wikidata.org/entity/Q76819'), ('wwname', 'Christoph Waltz')]
[('ww', 'http://www.wikidata.org/entit

16

In [ ]:
# Now I count

In [2]:
queryString = """
SELECT DISTINCT  (count(distinct ?ww) as ?howmany)
WHERE {

{
SELECT DISTINCT  ?ww ?wwname
WHERE { 
 ?ww wdt:P106 ?proWA ;       # ?fd --> film directors
     wdt:P31  wd:Q5 .
   ?filmwa ?p  ?ww.  
{ 
SELECT DISTINCT  ?filmwa
WHERE { 

 ?filmwa wdt:P31 wd:Q11424 .    # ?filmwa --> film Woody Allen
 ?filmwa wdt:P57 wd:Q25089 .        
}
}  
# this returns the labels
    ?ww <http://schema.org/name> ?wwname .
   # ?proWA <http://schema.org/name> ?pronameWA .
}
}

{
SELECT DISTINCT  ?wq ?wqname
WHERE { 


 ?wq wdt:P106 ?proQT ;       # ? --> film directors
     wdt:P31  wd:Q5 .
   ?filmqt ?p  ?wq. 
{ 
SELECT DISTINCT  ?filmqt
WHERE { 

 ?filmqt wdt:P31 wd:Q11424 .    # ?filmqt --> film Quentin Tarantino
 ?filmqt wdt:P57 wd:Q3772.        
}
}  
# this returns the labels
    ?wq <http://schema.org/name> ?wqname .
   # ?proQT <http://schema.org/name> ?pronameQT .
}
}

FILTER(?wq = ?ww)
}


"""

print("Results")
run_query(queryString)

Results
[('howmany', '16')]


1

In [ ]:
# what is the film with the highest number of shared actors?

In [3]:
# GET: 'cast member' --> wdt:P161 (from above)

In [ ]:
# I found the number of actors shared

In [5]:
queryString = """
SELECT DISTINCT  (count(distinct ?ww) as ?howmany)
WHERE {

{
SELECT DISTINCT  ?ww ?wwname
WHERE { 
 ?ww wdt:P106 ?proWA ;       # ?fd --> film directors
     wdt:P31  wd:Q5 .
   ?filmwa  wdt:P161  ?ww.  
{ 
SELECT DISTINCT  ?filmwa
WHERE { 

 ?filmwa wdt:P31 wd:Q11424 .    # ?filmwa --> film Woody Allen
 ?filmwa wdt:P57 wd:Q25089 .        
}
}  
# this returns the labels
    ?ww <http://schema.org/name> ?wwname .
   # ?proWA <http://schema.org/name> ?pronameWA .
}
}

{
SELECT DISTINCT  ?wq ?wqname
WHERE { 


 ?wq wdt:P106 ?proQT ;       # ? --> film directors
     wdt:P31  wd:Q5 .
   ?filmqt wdt:P161   ?wq. 
{ 
SELECT DISTINCT  ?filmqt
WHERE { 

 ?filmqt wdt:P31 wd:Q11424 .    # ?filmqt --> film Quentin Tarantino
 ?filmqt wdt:P57 wd:Q3772.        
}
}  
# this returns the labels
    ?wq <http://schema.org/name> ?wqname .
   # ?proQT <http://schema.org/name> ?pronameQT .
}
}

FILTER(?wq = ?ww)
}


"""

print("Results")
run_query(queryString)

Results
[('howmany', '12')]


1

In [ ]:
# Now I find the films with actors shared

In [8]:
queryString = """
SELECT DISTINCT ?filmwaname ?filmqtname (count(distinct ?ww) as ?howmany)
WHERE {

{
SELECT DISTINCT  ?ww ?wwname ?filmwaname
WHERE { 
 ?ww wdt:P106 ?proWA ;       # ?fd --> film directors
     wdt:P31  wd:Q5 .
   ?filmwa  wdt:P161  ?ww.  
{ 
SELECT DISTINCT  ?filmwa
WHERE { 

 ?filmwa wdt:P31 wd:Q11424 .    # ?filmwa --> film Woody Allen
 ?filmwa wdt:P57 wd:Q25089 .        
}
}  
# this returns the labels
    ?ww <http://schema.org/name> ?wwname .
   # ?proWA <http://schema.org/name> ?pronameWA .
   ?filmwa <http://schema.org/name> ?filmwaname .
}
}

{
SELECT DISTINCT  ?wq ?wqname ?filmqtname
WHERE { 


 ?wq wdt:P106 ?proQT ;       # ? --> film directors
     wdt:P31  wd:Q5 .
   ?filmqt wdt:P161   ?wq. 
{ 
SELECT DISTINCT  ?filmqt
WHERE { 

 ?filmqt wdt:P31 wd:Q11424 .    # ?filmqt --> film Quentin Tarantino
 ?filmqt wdt:P57 wd:Q3772.        
}
}  
# this returns the labels
    ?wq <http://schema.org/name> ?wqname .
   # ?proQT <http://schema.org/name> ?pronameQT .
    ?filmqt <http://schema.org/name> ?filmqtname .
}
}

FILTER(?wq = ?ww)
}
ORDER BY DESC (?howmany)

LIMIT 1

"""

print("Results")
run_query(queryString)

Results
[('filmwaname', 'New York Stories'), ('filmqtname', 'Pulp Fiction'), ('howmany', '2')]


1

# Task 6

In [9]:
# 6. Is the maximum budget for a Tarantino's movie higher of the max budget of an Allen's movie?

In [ ]:
# To find the budget for a film, i think to find all numerical properties for a film

In [ ]:
# From Workload 1

In [2]:
queryString = """
SELECT DISTINCT ?p ?pname #?o
WHERE { 


 ?film wdt:P31 wd:Q11424 ;
       ?p ?o .
 

 
# this returns the labels
    ?p <http://schema.org/name> ?pname .
    
FILTER(isNumeric(?o))
    
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pname', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P2047'), ('pname', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P2130'), ('pname', 'cost')]
[('p', 'http://www.wikidata.org/prop/direct/P2142'), ('pname', 'box office')]
[('p', 'http://www.wikidata.org/prop/direct/P8687'), ('pname', 'social media followers')]
[('p', 'http://www.wikidata.org/prop/direct/P1104'), ('pname', 'number of pages')]
[('p', 'http://www.wikidata.org/prop/direct/P2769'), ('pname', 'budget')]
[('p', 'http://www.wikidata.org/prop/direct/P1110'), ('pname', 'attendance')]
[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('pname', 'number of seasons')]
[('p', 'http://www.wikidata.org/prop/direct/P5436'), ('pname', 'number of viewers/listeners')]


10

In [ ]:
# GET: 'budget' --> wdt:P2769

In [ ]:
# wd:Q25089	Woody Allen
# wd:Q3772	Quentin Tarantino

In [ ]:
queryString = """
SELECT DISTINCT  ?fdname ?filmname ?b
WHERE { 

 ?film wdt:P57 ?fd .
 ?film wdt:P2769 ?b .                # ?b --> budget film
 

    ?fd <http://schema.org/name> ?fdname .
    ?film <http://schema.org/name> ?filmname .
    
FILTER(?fd = wd:Q25089 or ?fd = wd:Q3772 )
    
}

ORDER BY DESC (?b)
LIMIT 10

"""

print("Results")
run_query(queryString)

# Task 7

In [ ]:
# Who has films with more nominations for Academy Awards and who won more Academy Awards 
# (with his films not only personal awards).

## Task 7.1

In [9]:
# 7.1 Find the BGP for Academy Awards

In [10]:
# Now i want to find in Woddy Allen properties, something linked to 'awards'

In [12]:
queryString = """
SELECT DISTINCT  ?p ?pname
WHERE { 

wd:Q25089 ?p ?o .

# this returns the labels
    ?p <http://schema.org/name> ?pname .

FILTER REGEX(?pname, "award")
}

ORDER BY ASC (?pname)
LIMIT 20

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]


1

In [ ]:
# GET: 'award received' --> wdt:P166 

In [ ]:
# Use to find awards and analyze it

In [14]:
queryString = """
SELECT DISTINCT  ?o ?oname
WHERE { 

wd:Q25089 wdt:P166  ?o .      # ?o --> award

# this returns the labels
    ?o <http://schema.org/name> ?oname .


}


LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3404523'), ('oname', 'David di Donatello for Best Foreign Actor')]
[('o', 'http://www.wikidata.org/entity/Q31579'), ('oname', 'David di Donatello')]
[('o', 'http://www.wikidata.org/entity/Q3241784'), ('oname', 'Golden Lion for Lifetime Achievements')]
[('o', 'http://www.wikidata.org/entity/Q50616934'), ('oname', 'honorary doctorate of Pompeu Fabra University')]
[('o', 'http://www.wikidata.org/entity/Q3404587'), ('oname', 'FIPRESCI Prize of the Festival de Cannes')]
[('o', 'http://www.wikidata.org/entity/Q3822438'), ('oname', 'Butaca Awards')]
[('o', 'http://www.wikidata.org/entity/Q103360'), ('oname', 'Academy Award for Best Director')]
[('o', 'http://www.wikidata.org/entity/Q13452531'), ('oname', 'Commandeur des Arts et des Lettres\u200e')]
[('o', 'http://www.wikidata.org/entity/Q139184'), ('oname', 'BAFTA Award for Best Film')]
[('o', 'http://www.wikidata.org/entity/Q174389'), ('oname', 'César Award')]


10

In [ ]:
# GET: 'Academy Award for Best Director' --> wd:Q103360
# Find his instance

In [15]:
queryString = """
SELECT DISTINCT  ?o ?oname
WHERE { 

wd:Q103360 wdt:P31  ?o .     

# this returns the labels
    ?o <http://schema.org/name> ?oname .


}


LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q96474679'), ('oname', 'award for best direction')]
[('o', 'http://www.wikidata.org/entity/Q19020'), ('oname', 'Academy Awards')]


2

In [ ]:
# GET: 'Academy Awards' --> wd:Q19020  (exactly what I was looking for)

In [ ]:
# Find all Academy Awards

In [2]:
queryString = """
SELECT DISTINCT  ?awards ?awardsname
WHERE { 

?awards wdt:P31  wd:Q19020  .     

# this returns the labels
    ?awards <http://schema.org/name> ?awardsname .


}


LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('awards', 'http://www.wikidata.org/entity/Q22235305'), ('awardsname', 'Academy Award for Best Original Musical or Comedy Score')]
[('awards', 'http://www.wikidata.org/entity/Q21995136'), ('awardsname', 'Academy Award for Best Cinematography, Color')]
[('awards', 'http://www.wikidata.org/entity/Q21995139'), ('awardsname', 'Academy Award for Best Cinematography, Black-and-White')]
[('awards', 'http://www.wikidata.org/entity/Q22253131'), ('awardsname', 'Academy Award for Best Art Direction, Color')]
[('awards', 'http://www.wikidata.org/entity/Q22752811'), ('awardsname', 'Academy Award for Best Score, Adaptation or Treatment')]
[('awards', 'http://www.wikidata.org/entity/Q8610'), ('awardsname', 'Academy Scientific and Technical Award')]
[('awards', 'http://www.wikidata.org/entity/Q1324407'), ('awardsname', 'Academy Award for Best Live Action Short Film')]
[('awards', 'http://www.wikidata.org/entity/Q487136'), ('awardsname', 'Academy Award for Best Makeup and Hairstyling')]
[('awa

10

## Task 7.2

In [ ]:
# 7.2 Find the related subproperties

In [6]:
queryString = """
SELECT DISTINCT  ?awards  ?sp ?spname
WHERE { 

 ?awards wdt:P31  wd:Q19020 .
  ?sp   wdt:P1647* ?awards.

# this returns the labels
    ?awards <http://schema.org/name> ?awardsname .
?sp <http://schema.org/name> ?spname .

}


LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('awards', 'http://www.wikidata.org/entity/Q22235305'), ('sp', 'http://www.wikidata.org/entity/Q22235305'), ('spname', 'Academy Award for Best Original Musical or Comedy Score')]
[('awards', 'http://www.wikidata.org/entity/Q21995136'), ('sp', 'http://www.wikidata.org/entity/Q21995136'), ('spname', 'Academy Award for Best Cinematography, Color')]
[('awards', 'http://www.wikidata.org/entity/Q21995139'), ('sp', 'http://www.wikidata.org/entity/Q21995139'), ('spname', 'Academy Award for Best Cinematography, Black-and-White')]
[('awards', 'http://www.wikidata.org/entity/Q22253131'), ('sp', 'http://www.wikidata.org/entity/Q22253131'), ('spname', 'Academy Award for Best Art Direction, Color')]
[('awards', 'http://www.wikidata.org/entity/Q22752811'), ('sp', 'http://www.wikidata.org/entity/Q22752811'), ('spname', 'Academy Award for Best Score, Adaptation or Treatment')]
[('awards', 'http://www.wikidata.org/entity/Q8610'), ('sp', 'http://www.wikidata.org/entity/Q8610'), ('spname', 'Acade

10

## Task 7.3

In [ ]:
# 7.3 Find how they are related to the directors

In [4]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?awards wdt:P31  wd:Q19020  .     


?fd wdt:P106 wd:Q2526255 ;       # ?fd --> film directors
     wdt:P31  wd:Q5 ;
     ?p ?awards .
 

 
# this returns the labels
   
    ?fd <http://schema.org/name> ?fdname .
    ?awards <http://schema.org/name> ?awardsname .
    ?p <http://schema.org/name> ?pname .
}


LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]


2

## Task 7.4

In [ ]:
# 7.4 Are there alternative queries to get the same result?

In [7]:
# Who has films with more nominations for Academy Awards and who won more Academy Awards (with his films 
# not only personal awards).

In [ ]:
# Look now for all directors that direct a film and his nominations

In [8]:
queryString = """
SELECT DISTINCT  ?fd ?fdname ?filmname ?nominame
WHERE { 

 
 ?win wdt:P31  wd:Q19020  .
 
 ?film wdt:P31 wd:Q11424 .
  
 ?fd wdt:P106 wd:Q2526255 ;       # ?fd --> film directors
     wdt:P31  wd:Q5 .
 
 ?film wdt:P57 ?fd ;
       wdt:P1411 ?nomi .
     
 
 
        
 
 
# this returns the labels
    ?film <http://schema.org/name> ?filmname .

 
# this returns the labels
   
    ?fd <http://schema.org/name> ?fdname .
    
    ?nomi <http://schema.org/name> ?nominame .
}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('fd', 'http://www.wikidata.org/entity/Q2052126'), ('fdname', 'Pare Lorentz'), ('filmname', 'The Fight for Life'), ('nominame', 'Academy Award for Best Original Score')]
[('fd', 'http://www.wikidata.org/entity/Q17582678'), ('fdname', 'Yervant Gianikian'), ('filmname', "Angela's Diaries"), ('nominame', 'IFFR audience award')]
[('fd', 'http://www.wikidata.org/entity/Q17582684'), ('fdname', 'Angela Ricci Lucchi'), ('filmname', "Angela's Diaries"), ('nominame', 'IFFR audience award')]
[('fd', 'http://www.wikidata.org/entity/Q6290103'), ('fdname', 'Joshua Oppenheimer'), ('filmname', 'The Look of Silence'), ('nominame', 'European Film Award for Best Documentary')]
[('fd', 'http://www.wikidata.org/entity/Q6290103'), ('fdname', 'Joshua Oppenheimer'), ('filmname', 'The Act of Killing'), ('nominame', 'European Film Award for Best Documentary')]
[('fd', 'http://www.wikidata.org/entity/Q6290103'), ('fdname', 'Joshua Oppenheimer'), ('filmname', 'The Look of Silence'), ('nominame', 'Academy

10

In [ ]:
# Now count the nominations for films linked to director

In [7]:
queryString = """
SELECT DISTINCT  ?fd ?fdname (count(distinct ?nominame) as ?num_film_nominations)
WHERE { 


 ?win wdt:P31  wd:Q19020  .
 ?film wdt:P31 wd:Q11424 .
  
 ?fd wdt:P106 wd:Q2526255 ;       # ?fd --> film directors
     wdt:P31  wd:Q5 .
 
 ?film wdt:P57 ?fd ;
       wdt:P1411 ?nomi .
     
 
 
        
 
 
# this returns the labels
    ?film <http://schema.org/name> ?filmname .

 
# this returns the labels
   
    ?fd <http://schema.org/name> ?fdname .
    
    ?nomi <http://schema.org/name> ?nominame .
}

ORDER BY DESC (?num_film_nominations)
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('fd', 'http://www.wikidata.org/entity/Q55258'), ('fdname', 'Stephen Frears'), ('num_film_nominations', '93')]
[('fd', 'http://www.wikidata.org/entity/Q193628'), ('fdname', 'Sofia Coppola'), ('num_film_nominations', '81')]
[('fd', 'http://www.wikidata.org/entity/Q295964'), ('fdname', 'Jon Favreau'), ('num_film_nominations', '73')]
[('fd', 'http://www.wikidata.org/entity/Q55424'), ('fdname', 'Peter Weir'), ('num_film_nominations', '59')]
[('fd', 'http://www.wikidata.org/entity/Q314342'), ('fdname', 'David O. Russell'), ('num_film_nominations', '54')]
[('fd', 'http://www.wikidata.org/entity/Q3772'), ('fdname', 'Quentin Tarantino'), ('num_film_nominations', '50')]
[('fd', 'http://www.wikidata.org/entity/Q230448'), ('fdname', 'Jane Campion'), ('num_film_nominations', '47')]
[('fd', 'http://www.wikidata.org/entity/Q48987'), ('fdname', 'Jonathan Demme'), ('num_film_nominations', '46')]
[('fd', 'http://www.wikidata.org/entity/Q103646'), ('fdname', 'Ron Howard'), ('num_film_nomination

10

In [ ]:
# Now count the winner for films linked to director

In [6]:
queryString = """
SELECT DISTINCT  ?fd ?fdname (count(?winame) as ?num_film_winners)
WHERE { 

 ?win wdt:P31  wd:Q19020  .
 ?film wdt:P31 wd:Q11424 .
  
 ?fd wdt:P106 wd:Q2526255 ;       # ?fd --> film directors
     wdt:P31  wd:Q5 .
 
 ?film wdt:P57 ?fd ;
       wdt:P166 ?win .
     
 
 
        
 
 
# this returns the labels
    ?film <http://schema.org/name> ?filmname .

 
# this returns the labels
   
    ?fd <http://schema.org/name> ?fdname .
    
    ?win <http://schema.org/name> ?winame .
}

ORDER BY DESC (?num_film_winners)
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('fd', 'http://www.wikidata.org/entity/Q51495'), ('fdname', 'William Wyler'), ('num_film_winners', '41')]
[('fd', 'http://www.wikidata.org/entity/Q8877'), ('fdname', 'Steven Spielberg'), ('num_film_winners', '32')]
[('fd', 'http://www.wikidata.org/entity/Q56014'), ('fdname', 'George Cukor'), ('num_film_winners', '30')]
[('fd', 'http://www.wikidata.org/entity/Q55260'), ('fdname', 'David Lean'), ('num_film_winners', '30')]
[('fd', 'http://www.wikidata.org/entity/Q55420'), ('fdname', 'Fred Zinnemann'), ('num_film_winners', '23')]
[('fd', 'http://www.wikidata.org/entity/Q51114'), ('fdname', 'John Ford'), ('num_film_winners', '21')]
[('fd', 'http://www.wikidata.org/entity/Q72717'), ('fdname', 'Elia Kazan'), ('num_film_winners', '21')]
[('fd', 'http://www.wikidata.org/entity/Q51535'), ('fdname', 'Vincente Minnelli'), ('num_film_winners', '21')]
[('fd', 'http://www.wikidata.org/entity/Q4465'), ('fdname', 'Peter Jackson'), ('num_film_winners', '20')]
[('fd', 'http://www.wikidata.org/e

10

In [17]:
# Now count the nominations for director

In [5]:
queryString = """
SELECT DISTINCT  ?fd ?fdname (count(distinct ?nominame) as ?num_nominations)
WHERE { 

 ?nomi wdt:P31  wd:Q19020  .

 ?fd wdt:P106 wd:Q2526255 ;       # ?fd --> film directors
     wdt:P31  wd:Q5 ;
     wdt:P1411 ?nomi .


 
# this returns the labels
   
    ?fd <http://schema.org/name> ?fdname .
    
    ?nomi <http://schema.org/name> ?nominame .
}

ORDER BY DESC (?num_nominations)
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('fd', 'http://www.wikidata.org/entity/Q313281'), ('fdname', 'Michel Legrand'), ('num_nominations', '6')]
[('fd', 'http://www.wikidata.org/entity/Q8704'), ('fdname', 'Walt Disney'), ('num_nominations', '6')]
[('fd', 'http://www.wikidata.org/entity/Q23844'), ('fdname', 'George Clooney'), ('num_nominations', '6')]
[('fd', 'http://www.wikidata.org/entity/Q55294'), ('fdname', 'Kenneth Branagh'), ('num_nominations', '5')]
[('fd', 'http://www.wikidata.org/entity/Q51547'), ('fdname', 'Billy Wilder'), ('num_nominations', '5')]
[('fd', 'http://www.wikidata.org/entity/Q51575'), ('fdname', 'John Huston'), ('num_nominations', '5')]
[('fd', 'http://www.wikidata.org/entity/Q13595531'), ('fdname', 'Ethan Coen'), ('num_nominations', '5')]
[('fd', 'http://www.wikidata.org/entity/Q2001'), ('fdname', 'Stanley Kubrick'), ('num_nominations', '5')]
[('fd', 'http://www.wikidata.org/entity/Q28028'), ('fdname', 'Alfonso Cuarón'), ('num_nominations', '5')]
[('fd', 'http://www.wikidata.org/entity/Q95030

10

In [ ]:
# Now count the winners for director

In [4]:
queryString = """
SELECT DISTINCT  ?fd ?fdname (count(distinct ?winname) as ?num_winners)
WHERE { 


 ?win wdt:P31  wd:Q19020  .

 ?fd wdt:P106 wd:Q2526255 ;       # ?fd --> film directors
     wdt:P31  wd:Q5 ;
     wdt:P166 ?win .


 
# this returns the labels
   
    ?fd <http://schema.org/name> ?fdname .
    
    ?win <http://schema.org/name> ?winname .
}

ORDER BY DESC (?num_winners)
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('fd', 'http://www.wikidata.org/entity/Q8704'), ('fdname', 'Walt Disney'), ('num_winners', '7')]
[('fd', 'http://www.wikidata.org/entity/Q51547'), ('fdname', 'Billy Wilder'), ('num_winners', '4')]
[('fd', 'http://www.wikidata.org/entity/Q13595531'), ('fdname', 'Ethan Coen'), ('num_winners', '4')]
[('fd', 'http://www.wikidata.org/entity/Q13595311'), ('fdname', 'Joel Coen'), ('num_winners', '4')]
[('fd', 'http://www.wikidata.org/entity/Q56094'), ('fdname', 'Francis Ford Coppola'), ('num_winners', '4')]
[('fd', 'http://www.wikidata.org/entity/Q42574'), ('fdname', 'James Cameron'), ('num_winners', '3')]
[('fd', 'http://www.wikidata.org/entity/Q1999598'), ('fdname', 'Richard Williams'), ('num_winners', '3')]
[('fd', 'http://www.wikidata.org/entity/Q59085'), ('fdname', 'Robert Benton'), ('num_winners', '3')]
[('fd', 'http://www.wikidata.org/entity/Q43203'), ('fdname', 'Clint Eastwood'), ('num_winners', '3')]
[('fd', 'http://www.wikidata.org/entity/Q95125'), ('fdname', 'James L. Broo

10